# Improved Network Intrusion Detection using Deep Learning

This notebook contains improved implementations of CNN, LSTM, and CNN-LSTM models for network intrusion detection on NSL-KDD dataset.

## Improvements:
1. **Bug Fixes**: Fixed typo in column alignment
2. **Enhanced Architectures**: Added BatchNormalization, improved regularization
3. **Class Imbalance Handling**: Implemented class weights for balanced training
4. **Advanced Training**: Early stopping, learning rate scheduling, model checkpointing
5. **Better Evaluation**: ROC-AUC curves, classification reports, enhanced visualizations
6. **Code Quality**: Better error handling and modular design


In [ ]:
# Cell 1 — Setup: installs, imports, seed, GPU check, constants
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.regularizers import l2

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    confusion_matrix, matthews_corrcoef, roc_auc_score, 
    roc_curve, classification_report
)


In [ ]:
# Reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# Aggressively disable XLA JIT compilation to avoid shape compatibility issues
tf.config.optimizer.set_jit(False)
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'
os.environ['TF_DISABLE_XLA'] = '1'
# Disable XLA for all operations
tf.config.experimental.enable_op_determinism()

print("TF version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))

# Mixed precision disabled to avoid shape compatibility issues
# If needed, can be enabled but requires careful handling of dtypes
# try:
#     policy = tf.keras.mixed_precision.Policy('mixed_float16')
#     tf.keras.mixed_precision.set_global_policy(policy)
#     print("Mixed precision enabled")
# except:
#     print("Mixed precision not available")
print("Using float32 precision (mixed precision disabled for stability)")
print("XLA JIT compilation disabled to avoid shape compatibility issues")


In [ ]:
# Constants
RESULTS_DIR = "./results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Training parameters
EPOCHS = 50
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.2
PATIENCE = 10  # For early stopping

sns.set(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (12, 8)
print("Setup complete.")


In [ ]:
# Cell 2 — NSL-KDD loading and preprocessing
# Using direct file paths - update these to match your dataset location
train_fp = "/kaggle/input/nslkdd/KDDTrain+.txt"
test_fp = "/kaggle/input/nslkdd/KDDTest+.txt"

# Alternative paths for local execution
if not os.path.exists(train_fp):
    train_fp = "./KDDTrain+.txt"
    test_fp = "./KDDTest+.txt"

print("Using NSL-KDD files:")
print(f"  Train: {train_fp}")
print(f"  Test: {test_fp}")


In [ ]:
# Read the data (NSL-KDD has no headers)
try:
    df_train = pd.read_csv(train_fp, header=None)
    df_test = pd.read_csv(test_fp, header=None)
    print("Data loaded successfully!")
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Please ensure the dataset files are in the correct location.")
    raise


In [ ]:
# Add column names based on NSL-KDD documentation
columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 
    'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 
    'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 
    'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 
    'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 
    'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 
    'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack_type', 'difficulty'
]

df_train.columns = columns
df_test.columns = columns

print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)
print("\nFirst few rows:")
print(df_train.head())


In [ ]:
# Convert to binary labels (normal = 0, attack = 1)
df_train['label'] = df_train['attack_type'].apply(lambda x: 0 if x == 'normal' else 1)
df_test['label'] = df_test['attack_type'].apply(lambda x: 0 if x == 'normal' else 1)

print("Label distribution - Train:")
train_dist = df_train['label'].value_counts()
print(train_dist)
print(f"\nTrain class ratio (Normal:Attack): {train_dist[0]/train_dist[1]:.2f}")

print("\nLabel distribution - Test:")
test_dist = df_test['label'].value_counts()
print(test_dist)
print(f"\nTest class ratio (Normal:Attack): {test_dist[0]/test_dist[1]:.2f}")

# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
train_dist.plot(kind='bar', ax=axes[0], title='Train Set Class Distribution', color=['skyblue', 'coral'])
axes[0].set_xlabel('Class (0=Normal, 1=Attack)')
axes[0].set_ylabel('Count')
test_dist.plot(kind='bar', ax=axes[1], title='Test Set Class Distribution', color=['skyblue', 'coral'])
axes[1].set_xlabel('Class (0=Normal, 1=Attack)')
axes[1].set_ylabel('Count')
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/class_distribution.png', dpi=150)
plt.show()

# Drop unnecessary columns
df_train = df_train.drop(['attack_type', 'difficulty'], axis=1)
df_test = df_test.drop(['attack_type', 'difficulty'], axis=1)


In [ ]:
# Handle categorical columns (protocol_type, service, flag)
categorical_columns = ['protocol_type', 'service', 'flag']

# One-hot encode categorical variables
df_train = pd.get_dummies(df_train, columns=categorical_columns, prefix=categorical_columns)
df_test = pd.get_dummies(df_test, columns=categorical_columns, prefix=categorical_columns)

# Align columns (some services might be missing in test set)
# FIXED BUG: Changed df_test.colum to df_test.columns
train_cols = df_train.columns
test_cols = df_test.columns

# Add missing columns to test set
missing_cols = set(train_cols) - set(test_cols)
if missing_cols:
    print(f"Adding {len(missing_cols)} missing columns to test set")
    for col in missing_cols:
        if col != 'label':
            df_test[col] = 0

# Remove extra columns from test set
extra_cols = set(test_cols) - set(train_cols)
if extra_cols:
    print(f"Removing {len(extra_cols)} extra columns from test set")
    df_test = df_test.drop(columns=list(extra_cols))

# Reorder test columns to match train
df_test = df_test[train_cols]

print(f"\nAfter preprocessing:")
print(f"  Train shape: {df_train.shape}")
print(f"  Test shape: {df_test.shape}")
print(f"  Number of features: {len(train_cols) - 1}")


In [ ]:
# Prepare features and labels
feature_cols = [col for col in df_train.columns if col != 'label']

# Scale numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(df_train[feature_cols].astype(float))
X_test = scaler.transform(df_test[feature_cols].astype(float))

# Calculate class weights for imbalanced dataset
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(df_train['label']),
    y=df_train['label']
)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print(f"Class weights: {class_weight_dict}")

# Reshape for CNN/LSTM (samples, timesteps, features)
timesteps = X_train.shape[1]
X_train = X_train.reshape((-1, timesteps, 1))
X_test = X_test.reshape((-1, timesteps, 1))

y_train = df_train['label'].values
y_test = df_test['label'].values

print(f"\nFinal shapes:")
print(f"  X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"  y_train: {y_train.shape}, y_test: {y_test.shape}")
print(f"  Timesteps: {timesteps}")


In [ ]:
# Cell 3 — Improved Model definitions
# Based on original architectures with safe enhancements

def build_cnn_improved(input_shape):
    """Enhanced CNN model - based on original with regularization"""
    model = keras.Sequential([
        layers.Conv1D(64, 3, activation='relu', input_shape=input_shape),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        layers.Dropout(0.3),
        layers.Dense(2, activation='softmax')
    ])
    return model

def build_lstm_improved(input_shape):
    """Enhanced LSTM model - based on original with bidirectional"""
    model = keras.Sequential([
        layers.Bidirectional(layers.LSTM(64), input_shape=input_shape),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        layers.Dense(2, activation='softmax')
    ])
    return model

def build_cnn_lstm_improved(input_shape):
    """Enhanced CNN-LSTM hybrid - based on original architecture"""
    model = keras.Sequential([
        layers.Conv1D(64, 3, activation='relu', input_shape=input_shape),
        layers.MaxPooling1D(2),
        layers.LSTM(64),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        layers.Dense(2, activation='softmax')
    ])
    return model

print("Model architectures defined successfully!")


In [ ]:
# Cell 4 — Enhanced Training and Evaluation Functions

def create_callbacks(model_name):
    """Create callbacks for training"""
    callback_list = [
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=PATIENCE,
            restore_best_weights=True,
            verbose=1
        ),
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        callbacks.ModelCheckpoint(
            filepath=f'{RESULTS_DIR}/{model_name}_best.h5',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        ),
        callbacks.CSVLogger(f'{RESULTS_DIR}/{model_name}_training.log')
    ]
    return callback_list

def train_model_improved(model, name, class_weight=None):
    """Enhanced training function with callbacks and class weights"""
    # Disable XLA JIT compilation to avoid shape compatibility issues
    tf.config.optimizer.set_jit(False)
    
    # Use Adam optimizer with learning rate scheduling
    initial_lr = 0.001
    optimizer = keras.optimizers.Adam(learning_rate=initial_lr)
    
    # Use only accuracy metric to avoid shape compatibility issues with precision/recall
    # Run eagerly to bypass graph compilation issues
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
        run_eagerly=False  # Set to True if still having issues
    )
    
    print(f"\n{'='*60}")
    print(f"Training {name}")
    print(f"{'='*60}")
    print(f"Model summary:")
    model.summary()
    
    callback_list = create_callbacks(name)
    
    # Test model with a small batch first to catch shape errors early
    try:
        # Test forward pass
        test_input = X_train[:1]
        _ = model(test_input, training=False)
        print("Model forward pass test successful!")
    except Exception as e:
        print(f"ERROR: Model forward pass failed: {e}")
        print("This indicates a shape mismatch in the model architecture.")
        raise
    
    history = model.fit(
        X_train, y_train,
        validation_split=VALIDATION_SPLIT,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        class_weight=class_weight,
        callbacks=callback_list,
        verbose=1
    )
    
    return model, history

def evaluate_model_improved(model, name, history=None):
    """Enhanced evaluation with ROC curves and detailed metrics"""
    y_pred_proba = model.predict(X_test, verbose=0, batch_size=BATCH_SIZE)
    y_pred = np.argmax(y_pred_proba, axis=1)
    y_pred_proba_class1 = y_pred_proba[:, 1]  # Probability of attack class
    
    # Calculate metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, zero_division=0),
        'recall': recall_score(y_test, y_pred, zero_division=0),
        'f1': f1_score(y_test, y_pred, zero_division=0),
        'mcc': matthews_corrcoef(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_pred_proba_class1)
    }
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Create comprehensive visualization
    fig = plt.figure(figsize=(16, 5))
    
    # Plot 1: Confusion Matrix
    ax1 = plt.subplot(1, 3, 1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Normal', 'Attack'], 
                yticklabels=['Normal', 'Attack'],
                ax=ax1)
    ax1.set_title(f'Confusion Matrix - {name}')
    ax1.set_ylabel('True Label')
    ax1.set_xlabel('Predicted Label')
    
    # Plot 2: ROC Curve
    ax2 = plt.subplot(1, 3, 2)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_class1)
    ax2.plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {metrics["roc_auc"]:.4f})')
    ax2.plot([0, 1], [0, 1], 'k--', linewidth=1)
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title(f'ROC Curve - {name}')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Training History (if available)
    if history is not None:
        ax3 = plt.subplot(1, 3, 3)
        ax3.plot(history.history['loss'], label='Train Loss', linewidth=2)
        ax3.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
        ax3.set_xlabel('Epoch')
        ax3.set_ylabel('Loss')
        ax3.set_title(f'Training History - {name}')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/{name}_evaluation.png', dpi=150, bbox_inches='tight')
    plt.close()
    
    # Print detailed results
    print(f"\n{name} Results:")
    print("-" * 60)
    for metric, value in metrics.items():
        print(f"  {metric.upper():15s}: {value:.4f}")
    
    # Classification report
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred, 
                                target_names=['Normal', 'Attack'],
                                digits=4))
    
    return metrics, history


In [ ]:
# Cell 5 — Run experiments with improved models
models = {
    'CNN_Improved': build_cnn_improved((timesteps, 1)),
    'LSTM_Improved': build_lstm_improved((timesteps, 1)),
    'CNN_LSTM_Improved': build_cnn_lstm_improved((timesteps, 1))
}

print("Starting model training with improvements...")
print(f"Training parameters:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Validation split: {VALIDATION_SPLIT}")
print(f"  Early stopping patience: {PATIENCE}")
print(f"  Class weights: {class_weight_dict}")

results = []
histories = {}

for name, model in models.items():
    print(f"\n{'='*60}")
    print(f"Processing {name}")
    print(f"{'='*60}")
    
    trained_model, history = train_model_improved(model, name, class_weight=class_weight_dict)
    metrics, _ = evaluate_model_improved(trained_model, name, history)
    metrics['model'] = name
    results.append(metrics)
    histories[name] = history
    
    print(f"\n{name} training completed!")
    print(f"{'='*60}\n")


In [ ]:
# Cell 6 — Comprehensive Results Analysis and Visualization

print(f"\n{'='*60}")
print("FINAL RESULTS SUMMARY")
print(f"{'='*60}\n")

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# Save results
results_df.to_csv(f'{RESULTS_DIR}/nsl_kdd_results_improved.csv', index=False)
print(f"\nResults saved to: {RESULTS_DIR}/nsl_kdd_results_improved.csv")

# Create comprehensive comparison plots
fig = plt.figure(figsize=(18, 12))

# Plot 1: Metrics Comparison Bar Chart
ax1 = plt.subplot(2, 3, 1)
metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
x = np.arange(len(results_df))
width = 0.15

for i, metric in enumerate(metrics_to_plot):
    offset = (i - len(metrics_to_plot)/2) * width + width/2
    ax1.bar(x + offset, results_df[metric], width, label=metric.replace('_', ' ').title())

ax1.set_xlabel('Models')
ax1.set_ylabel('Score')
ax1.set_title('Model Performance Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(results_df['model'], rotation=45, ha='right')
ax1.set_ylim(0, 1.05)
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: MCC Comparison
ax2 = plt.subplot(2, 3, 2)
bars = ax2.bar(results_df['model'], results_df['mcc'], color=['#3498db', '#e74c3c', '#2ecc71'])
ax2.set_ylabel('MCC Score')
ax2.set_title('Matthews Correlation Coefficient')
ax2.set_ylim(0, 1)
ax2.grid(True, alpha=0.3, axis='y')
for i, (bar, val) in enumerate(zip(bars, results_df['mcc'])):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{val:.4f}', ha='center', va='bottom')

# Plot 3: ROC-AUC Comparison
ax3 = plt.subplot(2, 3, 3)
bars = ax3.bar(results_df['model'], results_df['roc_auc'], color=['#9b59b6', '#f39c12', '#1abc9c'])
ax3.set_ylabel('ROC-AUC Score')
ax3.set_title('ROC-AUC Comparison')
ax3.set_ylim(0, 1)
ax3.grid(True, alpha=0.3, axis='y')
for i, (bar, val) in enumerate(zip(bars, results_df['roc_auc'])):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{val:.4f}', ha='center', va='bottom')

# Plot 4-6: Training History Comparison
for idx, (name, history) in enumerate(histories.items(), start=4):
    ax = plt.subplot(2, 3, idx)
    ax.plot(history.history['loss'], label='Train Loss', linewidth=2)
    ax.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    ax.plot(history.history['accuracy'], label='Train Acc', linewidth=2, linestyle='--')
    ax.plot(history.history['val_accuracy'], label='Val Acc', linewidth=2, linestyle='--')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Score')
    ax.set_title(f'{name} Training History')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/comprehensive_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# Find best model
best_model_idx = results_df['f1'].idxmax()
best_model = results_df.loc[best_model_idx, 'model']
print(f"\n{'='*60}")
print(f"BEST MODEL: {best_model}")
print(f"{'='*60}")
print(results_df.loc[best_model_idx].to_string())
print(f"{'='*60}")


In [ ]:
# Additional Analysis: Feature Importance and Model Comparison Table

print("\nDetailed Model Comparison:")
print("="*80)
comparison_table = results_df.set_index('model').T
print(comparison_table.to_string())
print("="*80)

# Save comparison table
comparison_table.to_csv(f'{RESULTS_DIR}/detailed_comparison.csv')

# Create a summary report
with open(f'{RESULTS_DIR}/summary_report.txt', 'w') as f:
    f.write("Network Intrusion Detection - Improved Model Results\n")
    f.write("="*80 + "\n\n")
    f.write(f"Dataset: NSL-KDD\n")
    f.write(f"Training samples: {len(y_train)}\n")
    f.write(f"Test samples: {len(y_test)}\n")
    f.write(f"Features: {timesteps}\n\n")
    f.write("Model Performance:\n")
    f.write("-"*80 + "\n")
    f.write(results_df.to_string(index=False))
    f.write("\n\n")
    f.write(f"Best Model (by F1-Score): {best_model}\n")
    f.write("="*80 + "\n")

print(f"\nSummary report saved to: {RESULTS_DIR}/summary_report.txt")
print("\nAll results and visualizations have been saved successfully!")
